In [21]:
import gmsh
import sys
import math
import json

# Initialize gmsh
gmsh.initialize()

# Choose kernel
model = gmsh.model
occ = model.occ
mesh = model.mesh
model.add("forma11c_gmsh")



bpp_thickness = 11e-3
width = 2e-3
gdl_thickness = 100e-6
mem_thickness = 50e-6
gas_channel_width = 1e-3
gas_channel_thickness = 1e-3

mesh_size_bpp = 15e-6
mesh_size_gdl = 10e-6
mesh_size_mem = 5e-6

points_bpp = [occ.addPoint(0, gdl_thickness+gas_channel_thickness+mem_thickness/2,0, meshSize=mesh_size_bpp),
              occ.addPoint(0, bpp_thickness+gdl_thickness+mem_thickness/2,0, meshSize=mesh_size_bpp),
              occ.addPoint(width, bpp_thickness+gdl_thickness+mem_thickness/2,0, meshSize=mesh_size_bpp),
              occ.addPoint(width, gdl_thickness+gas_channel_thickness+mem_thickness/2,0, meshSize=mesh_size_bpp)]

points_rib = [occ.addPoint(gas_channel_width, gdl_thickness+mem_thickness/2,0, meshSize=mesh_size_bpp),
              occ.addPoint(gas_channel_width, gdl_thickness+gas_channel_thickness+mem_thickness/2,0, meshSize=mesh_size_bpp),
              occ.addPoint(width, gdl_thickness+gas_channel_thickness+mem_thickness/2,0, meshSize=mesh_size_gdl),
              occ.addPoint(width, gdl_thickness+mem_thickness/2,0, meshSize=mesh_size_gdl)]


points_gdl = [occ.addPoint(0, mem_thickness/2,0, meshSize=mesh_size_gdl),
              occ.addPoint(0, gdl_thickness+mem_thickness/2,0, meshSize=mesh_size_gdl),
              occ.addPoint(gas_channel_width, gdl_thickness+mem_thickness/2,0, meshSize=mesh_size_gdl),
              occ.addPoint(width, gdl_thickness+mem_thickness/2,0, meshSize=mesh_size_gdl),
              occ.addPoint(width, mem_thickness/2,0, meshSize=mesh_size_gdl)]

points_mem = [occ.addPoint(0, 0,0, meshSize=mesh_size_mem),
              occ.addPoint(0, mem_thickness/2,0, meshSize=mesh_size_mem),
              occ.addPoint(width, mem_thickness/2,0, meshSize=mesh_size_mem),
              occ.addPoint(width, 0,0, meshSize=mesh_size_mem)]

channel_lines_bpp = [occ.addLine(points_bpp[i], points_bpp[i+1])
                 for i in range(-1, len(points_bpp)-1)]
channel_lines_rib = [occ.addLine(points_rib[i], points_rib[i+1])
                 for i in range(-1, len(points_rib)-1)]
channel_lines_gdl = [occ.addLine(points_gdl[i], points_gdl[i+1])
                 for i in range(-1, len(points_gdl)-1)]
channel_lines_mem = [occ.addLine(points_mem[i], points_mem[i+1])
                 for i in range(-1, len(points_mem)-1)]


In [22]:
channel_surface_bpp = occ.addCurveLoop(channel_lines_bpp,1)
channel_surface_bpp = occ.addCurveLoop(channel_lines_gdl,2)
channel_surface_bpp = occ.addCurveLoop(channel_lines_mem,3)
channel_surface_rib = occ.addCurveLoop(channel_lines_rib,4)

In [23]:
plane_surface_bpp = occ.addPlaneSurface([1],1) # add holes and other
plane_surface_rib = occ.addPlaneSurface([4],4)
plane_surface_gdl = occ.addPlaneSurface([2],2) 
plane_surface_mem = occ.addPlaneSurface([3],3) 

extrusion = occ.extrude([(2,1),(2,2),(2,3),(2,4)], 0, 0, 1e-3,numElements=[1], recombine=True)


In [24]:
bpp_rib = occ.fuse([(2,plane_surface_bpp)],[(2,plane_surface_rib)])
bpp_gdl = occ.fragment(bpp_rib[0],[(2,plane_surface_gdl)])   #(2,bpp_surface[0][0][1])
final_surface = occ.fragment(bpp_gdl[0],[(2,plane_surface_mem)])   #(2,bpp_surface[0][0][1])
occ.removeAllDuplicates()
occ.synchronize()

In [25]:
model.addPhysicalGroup(3,[1,4],1)
model.setPhysicalName(3,1,'BPP')
model.addPhysicalGroup(3,[2],2)
model.setPhysicalName(3,2,'GDL')
model.addPhysicalGroup(3,[3],3)
model.setPhysicalName(3,3,'MEM')

In [26]:
meshtype = 'Bump'
coef = 0.05
h_line = [95,80,74]
mesh.setTransfiniteCurve(h_line[0], 70,meshType = meshtype,coef=coef)
mesh.setTransfiniteCurve(h_line[1], 70,meshType = meshtype,coef=coef)
mesh.setTransfiniteCurve(h_line[2], 70,meshType = meshtype,coef=coef)


h_line_half = [86,89,68,64,101,103]

mesh.setTransfiniteCurve(h_line_half[0], 36,meshType = meshtype,coef=coef)
mesh.setTransfiniteCurve(h_line_half[1], 36,meshType = meshtype,coef=coef)
mesh.setTransfiniteCurve(h_line_half[2], 36,meshType = meshtype,coef=coef)
mesh.setTransfiniteCurve(h_line_half[3], 36,meshType = meshtype,coef=coef)
mesh.setTransfiniteCurve(h_line_half[4], 15,meshType = meshtype,coef=coef)
mesh.setTransfiniteCurve(h_line_half[5], 15,meshType = meshtype,coef=coef)


mem_vert = [97,99]
mesh.setTransfiniteCurve(mem_vert[0], 4,meshType = meshtype,coef=coef)
mesh.setTransfiniteCurve(mem_vert[1], 4,meshType = meshtype,coef=coef)

gdl_vert = [83,91]
mesh.setTransfiniteCurve(gdl_vert[0], 6,meshType = meshtype,coef=coef)
mesh.setTransfiniteCurve(gdl_vert[1], 6,meshType = meshtype,coef=coef)

bpp_vert = [71,76]
mesh.setTransfiniteCurve(bpp_vert[0], 100,meshType = meshtype,coef=coef)
mesh.setTransfiniteCurve(bpp_vert[1], 100,meshType = meshtype,coef=coef)

In [27]:
mesh.setTransfiniteSurface(occ.getEntities(2)[6][1],cornerTags=[42,43,45,46])
mesh.setTransfiniteSurface(occ.getEntities(2)[5][1],cornerTags=[1,44,3,35])
mesh.setTransfiniteSurface(occ.getEntities(2)[12][1],cornerTags=[39,10,36,38])
mesh.setTransfiniteSurface(occ.getEntities(2)[13][1],cornerTags=[50,51,55,48])

for i in range(len(occ.getEntities(2))):
    if i not in [5,6,12,13]:
        mesh.setTransfiniteSurface(occ.getEntities(2)[i][1])

for j in range(len(occ.getEntities(3))):
    mesh.setTransfiniteVolume(occ.getEntities(3)[j][1])

In [28]:
gmsh.option.setNumber('Mesh.QuadqsSizemapMethod', 4)
gmsh.option.setNumber('Mesh.RecombineAll', 1)
gmsh.option.setNumber('Mesh.RecombinationAlgorithm', 1)
gmsh.option.setNumber('Mesh.Recombine3DLevel', 2)
gmsh.option.setNumber('Mesh.ElementOrder', 1)
gmsh.option.setNumber('Mesh.Smoothing', 200)

In [29]:
#mesh.setRecombine(2, 1)
mesh.generate(3)
#mesh.optimize("Laplace2D")
mesh.optimize("Relocate3D")

In [30]:
gmsh.fltk.run()

In [31]:
gmsh.write("meshgmsh.msh")
gmsh.write("meshgmsh.vtk")
gmsh.clear()